In [1]:
# Import the dependencies.
import pandas as pd
import ipywidgets as widgets
import gmaps
import gmaps.datasets
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2022-04-21 15:41:57,42.6001,-73.9662,56.03,43,100,3.00
1,1,Port Alfred,ZA,2022-04-21 15:42:45,-33.5906,26.8910,77.18,52,70,2.84
2,2,Ushuaia,AR,2022-04-21 15:50:44,-54.8000,-68.3000,44.22,65,75,17.27
3,3,Yellowknife,CA,2022-04-21 15:52:14,62.4560,-114.3525,19.92,53,20,4.61
4,4,Busselton,AU,2022-04-21 15:42:41,-33.6500,115.3333,59.27,62,0,15.23


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Mapping cloudiness 

locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Mapping winds 

locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
print(f'you chose {min_temp} as a min temp and {max_temp} as your max temp')

you chose 75.0 as a min temp and 95.0 as your max temp


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Port Alfred,ZA,2022-04-21 15:42:45,-33.5906,26.8910,77.18,52,70,2.84
6,6,Padang,ID,2022-04-21 15:52:39,-0.9492,100.3543,78.84,83,94,2.77
10,10,Rikitea,PF,2022-04-21 15:43:49,-23.1203,-134.9692,77.22,78,98,19.13
18,18,Tautira,PF,2022-04-21 15:52:42,-17.7333,-149.1500,77.72,82,46,12.84
21,21,Hithadhoo,MV,2022-04-21 15:52:43,-0.6000,73.0833,84.56,66,8,7.96
24,24,Mahebourg,MU,2022-04-21 15:52:44,-20.4081,57.7000,79.25,83,75,9.22
26,26,Soc Trang,VN,2022-04-21 15:52:45,9.6033,105.9800,82.13,79,25,8.23
32,32,Novo Cruzeiro,BR,2022-04-21 15:52:47,-17.4681,-41.8753,80.40,42,15,4.79
35,35,Stephenville,US,2022-04-21 15:52:32,32.2207,-98.2023,79.25,68,75,16.11
44,44,Arraial Do Cabo,BR,2022-04-21 15:52:51,-22.9661,-42.0278,80.56,65,20,5.75


In [13]:
preferred_cities_df.count()

City_ID       182
City          182
Country       182
Date          182
Lat           182
Lng           182
Max Temp      182
Humidity      182
Cloudiness    182
Wind Speed    182
dtype: int64

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": '48.8566, 2.3522'}
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Port Alfred,ZA,77.18,-33.5906,26.8910,The Halyards Hotel
6,Padang,ID,78.84,-0.9492,100.3543,Mercure Padang
10,Rikitea,PF,77.22,-23.1203,-134.9692,People ThankYou
18,Tautira,PF,77.72,-17.7333,-149.1500,Pension Oaoa
21,Hithadhoo,MV,84.56,-0.6000,73.0833,Scoop Guest House
24,Mahebourg,MU,79.25,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
26,Soc Trang,VN,82.13,9.6033,105.9800,Phu Quy Hotel
32,Novo Cruzeiro,BR,80.40,-17.4681,-41.8753,Hotel Nossa Senhora das Graças
35,Stephenville,US,79.25,32.2207,-98.2023,La Quinta Inn & Suites by Wyndham Stephenville
44,Arraial Do Cabo,BR,80.56,-22.9661,-42.0278,Pousada Porto Praia
